In [40]:
import urllib.request
import datetime
import csv
import os
import glob
from pathlib import Path
from urllib.error import URLError, HTTPError
from datetime import timedelta  

BASE_URL = "http://realtime.bualuang.co.th/myeasy/realtime/quotation/txt/{}.txt"
STOCK_PATH = '/DEFINE-PATH-HERE/'

START_DATE = datetime.datetime.strptime('01012003', '%d%m%Y')

today = datetime.datetime.now()

should_fetch_from_start = not os.path.exists(STOCK_PATH)
           
def string_to_date(string, pattern = '%d%m%Y'):
    return datetime.datetime.strptime(string, pattern)

def fetch_stockenddate_price(date_obj, directory=STOCK_PATH):
    try:
        date_str = date_obj.strftime('%d%m%Y')
        response = urllib.request.urlopen(BASE_URL.format(date_str))
        data = response.read()
        text = data.decode('utf-8')
        f = open("{}{}.csv".format(directory,date_str),"w")
        f.write(text)
        f.close()
    except HTTPError as e:
        print("Market closed")
        
def create_base_directory(directory = STOCK_PATH):
    if not os.path.exists(directory):
        os.makedirs(directory)
        
def is_weekend(datetime):
    day = datetime.weekday()
    return day == 5 or day == 6

def get_latest_file(directory = STOCK_PATH):
    list_of_files = glob.glob('{}*.csv'.format(directory)) 
    if list_of_files:
        return max(list_of_files, key=os.path.getctime)
    
def fetch_stockenddate_price_until(date_to_fetch, date_to_stop, include=True):
    while date_to_fetch <= date_to_stop if include else date_to_fetch < date_to_stop:
        if not is_weekend(date_to_fetch):
            fetch_stockenddate_price(date_to_fetch)
        date_to_fetch = date_to_fetch + timedelta(days=1)   
        
def is_path_writable(path):
    return os.access(path, os.W_OK)

def fetch_from_start():
    create_base_directory()
    fetch_stockenddate_price_until(START_DATE, today)
    
if should_fetch_from_start:
    fetch_from_start()
else:
    fullpath = get_latest_file(STOCK_PATH)
    if fullpath:
       date_to_fetch = string_to_date(Path(fullpath).name.replace(".csv", '')) + timedelta(days=1)
       fetch_stockenddate_price_until(date_to_fetch, today)
    else:
        fetch_from_start()